## Package

In [3]:
import torch
import argparse
import os,glob
import librosa
import soundfile as sf
import numpy as np
from utils.hparams import HParam
import torch.multiprocessing as mp
from torch.multiprocessing import Pool, Process, set_start_method
from models.GPV import GPV

## Param

In [22]:
hp = HParam("../config/GPV_2.yaml")
chkpt= "/home/nas/user/kbh/VADK/chkpt/GPV_2/bestmodel.pt"

device = 'cuda:0'
torch.cuda.set_device(device)

n_mels =40
sr = 16000
n_fft = 512
n_shift = 128


num_epochs = 1
batch_size = 1

mel_basis = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)

path = '/home/nas/DB/[DB]AV-TR/[20211102] AVCV 시뮬레이션(도희준형)/2021-11-02_10-52-33.wav'

## Run

In [23]:
model = GPV(hp,inputdim=n_mels).to(device)

# Dirs
model.load_state_dict(torch.load(chkpt, map_location=device))
model.eval()

raw, _ = librosa.load(path,sr=sr)
display(np.shape(raw))
spec = librosa.stft(raw,window='hann',n_fft=n_fft,hop_length=n_shift, win_length=None,center=True,dtype=np.cdouble)
display(np.shape(spec))
mel = np.matmul(mel_basis,np.abs(spec))
display(np.shape(mel))
pt = torch.from_numpy(mel)
pt = pt.float()
pt = torch.unsqueeze(pt,dim=0)
pt = pt.cuda()

with torch.no_grad():
    output = model(pt)

(378880,)

(257, 2961)

(40, 2961)

In [18]:
output = torch.squeeze(output)
display(output.shape)

torch.Size([20343])

## STFT domain label -> wav label

In [19]:
thr = 0.5
p =0
o = raw
for idx in range(len(output)):
    if output[idx] < thr :
        p +=1
        o[(idx-1)*128:idx*128]=0

In [21]:
sf.write('out.wav',o,16000)